# 1. Setup and Library Import

In [1]:
import pandas as pd
import numpy as np
import re
import joblib
import nltk
from nltk.corpus import stopwords
from nltk.stem.isri import ISRIStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import ipywidgets as widgets
from IPython.display import display, clear_output
from datasets import load_dataset

nltk.download('stopwords', quiet=True)

True

# 2. Data Loading and Preprocessing

In [2]:
dataset = load_dataset("komari6/ajgt_twitter_ar")
df = pd.DataFrame(dataset['train'])
df.rename(columns={'label': 'Sentiment', 'text': 'Text'}, inplace=True)

stemmer = ISRIStemmer()
arabic_stopwords = set(stopwords.words('arabic'))
custom_stops = {'انا', 'انت', 'احنا', 'هم', 'لهم', 'لنا', 'في', 'من', 'على', 'عن', 'مع', 'اللي', 'شو', 'وين', 'كيف', 'ما', 'يا', 'بنا', 'اللهم'}
arabic_stopwords.update(custom_stops)

def clean_and_process(text):
    text = str(text)
    text = re.sub(r'[ًٌٍَُِّْ]', '', text)
    text = re.sub(r'[أإآ]', 'ا', text)
    text = re.sub(r'ة', 'ه', text)
    text = re.sub(r'ى', 'ي', text)
    text = re.sub(r'[^\u0600-\u06ff\s]', '', text)
    
    words = text.split()
    processed_words = [stemmer.stem(w) for w in words if w not in arabic_stopwords]
    return " ".join(processed_words)

df['Clean_Text'] = df['Text'].apply(clean_and_process)

# 3. Model Training (Logistic Regression)

In [3]:
tfidf = TfidfVectorizer(ngram_range=(1, 2), max_features=4000)
X = tfidf.fit_transform(df['Clean_Text']).toarray()
y = df['Sentiment'].apply(lambda x: 1 if str(x) in ['1', 'Positive'] else 0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

model = LogisticRegression(C=10, solver='liblinear', random_state=42)
model.fit(X_train, y_train)

print(f"Model Accuracy: {accuracy_score(y_test, model.predict(X_test)) * 100:.2f}%")

Model Accuracy: 86.39%


# 4. Feature Importance Analysis

In [4]:
feature_names = tfidf.get_feature_names_out()
coefs = model.coef_[0]
feature_importance = pd.DataFrame({'Word': feature_names, 'Weight': coefs})

print("Top Positive Words:")
print(feature_importance.sort_values(by='Weight', ascending=False).head(10)['Word'].values)

print("\nTop Negative Words:")
print(feature_importance.sort_values(by='Weight', ascending=True).head(10)['Word'].values)

Top Positive Words:
['الل' 'رزق' 'يرب' 'ربن' 'جعل' 'سلم' 'خير' 'رئع' 'صلح' 'جمل']

Top Negative Words:
['شعب' 'قرف' 'سخف' 'مش' 'سخر' 'جمل لحب' 'نوي' 'كلب' 'اخو' 'وطن']


# 5. Save Model Artifacts

In [5]:
joblib.dump(model, 'jo_senti_model.pkl')
joblib.dump(tfidf, 'tfidf_vectorizer.pkl')
print("Model and Vectorizer saved successfully.")

Model and Vectorizer saved successfully.


# 6. Interactive Application (Hybrid System)

In [8]:
loaded_model = joblib.load('jo_senti_model.pkl')
loaded_vectorizer = joblib.load('tfidf_vectorizer.pkl')

pos_keywords = {'كفو', 'نشمي', 'وحش', 'قدها', 'بشهي', 'بجنن', 'خرافي', 'فخم', 'رائع', 'زاكي', 'عفية', 'معلم', 'شيخ', 'مش بطال', 'يعطيك العافية', 'ما قصرت'}
neg_keywords = {'زفت', 'قرف', 'سيء', 'خزي', 'فاشل', 'حقير', 'كلب', 'سخيف', 'مش نافع', 'عدم المؤاخذة'}

header = widgets.HTML("<h2 style='color:#2c3e50'>Jo-Senti: Smart Hybrid App</h2>")
text_input = widgets.Textarea(placeholder='Type here...', layout=widgets.Layout(width='80%', height='100px'))
button = widgets.Button(description='Analyze', button_style='primary', icon='rocket')
output = widgets.Output()

def on_analyze(b):
    with output:
        clear_output()
        sentence = text_input.value
        if not sentence: return
        
        cleaned = clean_and_process(sentence)
        input_words = sentence.split()
        
        def check_keyword(keywords, text, words_list):
            for k in keywords:
                if ' ' in k:
                    if k in text: return True
                else:
                    if k in words_list: return True
            return False

        found_pos = check_keyword(pos_keywords, sentence, input_words)
        found_neg = check_keyword(neg_keywords, sentence, input_words)
        
        vec = loaded_vectorizer.transform([cleaned]).toarray()
        pred = loaded_model.predict(vec)[0]
        prob = loaded_model.predict_proba(vec)[0].max()
        
        if found_pos:
            res, color, src = "Positive 😍", "#d4edda", "Golden Phrase"
        elif found_neg:
            res, color, src = "Negative 😡", "#f8d7da", "Blacklist Phrase"
        else:
            if pred == 1:
                res, color, src = f"Positive 😊", "#e8f8f5", f"AI Model ({prob:.0%})"
            else:
                res, color, src = f"Negative 😔", "#fcf3cf", f"AI Model ({prob:.0%})"

        display(widgets.HTML(f"<div style='background-color:{color}; padding:15px; border-radius:10px;'><h3>Result: {res}</h3><p>Source: {src}</p></div>"))

button.on_click(on_analyze)
display(header, text_input, button, output)

HTML(value="<h2 style='color:#2c3e50'>Jo-Senti: Smart Hybrid App</h2>")

Textarea(value='', layout=Layout(height='100px', width='80%'), placeholder='Type here...')

Button(button_style='primary', description='Analyze', icon='rocket', style=ButtonStyle())

Output()